# ORIE 5256: Project
## By: Henry Ogworonjo (hco23), Yiqun Wang (yw2296), Nitish Gade (ng375)

---

## Import the relevant datasets, and split into features and target.

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report
import xgboost as xgb
import matplotlib.pyplot as plt
from numba import jit
from scipy.linalg import get_blas_funcs
from joblib import Parallel, delayed
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pickle
from tune_sklearn import TuneSearchCV
from tune_sklearn import TuneGridSearchCV
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("numerai_training_data.csv")
# There's 310 features
features = [c for c in df if c.startswith("feature")]
df["erano"] = df.era.str.slice(3).astype(int)
eras = df.erano
target = "target"
# The features are grouped together into 6 types
feature_groups = {
    g: [c for c in df if c.startswith(f"feature_{g}")]
    for g in ["intelligence", "wisdom", "charisma", "dexterity", "strength", "constitution"]
}

In [4]:
X = df[features].copy()
y = df[target].copy()

In [5]:
y = y*4
y = y.astype('int32')

0         2
1         1
2         1
3         1
4         3
         ..
501803    2
501804    3
501805    1
501806    2
501807    2
Name: target, Length: 501808, dtype: int32

In [6]:
dftourn = pd.read_csv("numerai_tournament_data.csv")

In [13]:
validation_set = dftourn[dftourn['data_type'] == 'validation']
V = validation_set.copy()
V = V[features]

In [14]:
y_test = validation_set['target']
y_test = y_test*4
y_test = y_test.astype('int32')

0          1
1          2
2          4
3          2
4          3
          ..
1545360    2
1545361    1
1545362    2
1545363    4
1545364    2
Name: target, Length: 137779, dtype: int32

In [24]:
test_set = dftourn
test_set = test_set.copy()
T = test_set[features]

---

## Correlation Analysis and PCA

In [10]:
Xcorr = X.corr()

,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,...,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46
feature_intelligence1,1.000000,-0.014157,-0.024404,0.652596,0.069868,0.151265,0.188467,0.760617,-0.017661,-0.029344,...,-0.086291,-0.088465,-0.061247,-0.120785,-0.120960,0.021476,0.037546,-0.125501,-0.032981,-0.168257
feature_intelligence2,-0.014157,1.000000,0.905315,-0.028097,0.184372,-0.048150,-0.031199,-0.014185,0.483491,0.465704,...,-0.135790,-0.127448,-0.141152,0.049052,-0.009823,-0.156142,-0.222864,0.025692,-0.044577,-0.109628
feature_intelligence3,-0.024404,0.905315,1.000000,-0.041086,0.173870,-0.053716,-0.027043,-0.027653,0.467432,0.503837,...,-0.119191,-0.105961,-0.134758,0.055136,0.001594,-0.152337,-0.232693,0.029589,-0.034833,-0.107264
feature_intelligence4,0.652596,-0.028097,-0.041086,1.000000,0.054492,0.103287,0.110541,0.815162,-0.030180,-0.043915,...,-0.080466,-0.096097,-0.114118,-0.110214,-0.107755,-0.007601,-0.000272,-0.130211,-0.036535,-0.209641
feature_intelligence5,0.069868,0.184372,0.173870,0.054492,1.000000,0.145064,0.154358,0.067219,0.190649,0.188392,...,-0.110536,-0.101133,-0.104007,-0.016642,-0.046638,-0.100245,-0.071022,-0.027973,-0.050636,-0.079726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
feature_wisdom42,0.021476,-0.156142,-0.152337,-0.007601,-0.100245,-0.043587,0.004963,-0.002595,-0.092348,-0.094492,...,0.708243,0.754589,0.841706,0.093309,0.171126,1.000000,0.311838,0.120180,0.263568,0.505521
feature_wisdom43,0.037546,-0.222864,-0.232693,-0.000272,-0.071022,-0.009765,0.030643,-0.004125,-0.276340,-0.288251,...,0.198455,0.196352,0.265656,-0.023005,0.081989,0.311838,1.000000,0.004611,0.002951,0.197166
feature_wisdom44,-0.125501,0.025692,0.029589,-0.130211,-0.027973,0.015334,0.010520,-0.139625,0.051455,0.032859,...,0.153831,0.161515,0.168145,0.905125,0.779072,0.120180,0.004611,1.000000,0.086558,0.172650
feature_wisdom45,-0.032981,-0.044577,-0.034833,-0.036535,-0.050636,-0.017144,-0.002331,-0.034896,0.080112,0.064880,...,0.303515,0.346918,0.299520,0.085849,0.100799,0.263568,0.002951,0.086558,1.000000,0.506321


In [11]:
def drop_correlated(mainmatrix,correlationmatrix,correlationthreshold,validationmatrix):
    correlated_features = set()
    for i in range(len(correlationmatrix.columns)):
        for j in range(i):
            if abs(correlationmatrix.iloc[i, j]) > correlationthreshold:
                colname = correlationmatrix.columns[i]
                correlated_features.add(colname)
    mainmatrix2 = mainmatrix.copy()
    mainmatrix2.drop(labels=correlated_features, axis=1, inplace=True)
    validationmatrix2 = validationmatrix.copy()
    validationmatrix2.drop(labels=correlated_features, axis=1, inplace=True)
    return mainmatrix2,validationmatrix2

In [12]:
def do_PCA(matrix,explainedvariance,validationmatrix):
    pca = PCA(n_components=matrix.shape[1])
    pca.fit(matrix)
    explainedvarianceratio = pca.explained_variance_ratio_
    cumulativevariance = np.cumsum(explainedvarianceratio)
    cutoff = np.searchsorted(cumulativevariance, explainedvariance, side='right')
    principalComponents = pca.fit_transform(matrix)
    principalComponents = principalComponents[:,:cutoff]
    principalDf = pd.DataFrame(data = principalComponents)
    X2_new = principalDf
    newcolumns = []
    for i in range(X2_new.shape[1]):
        string = 'feature' + str(i+1)
        newcolumns.append(string)
    X2_new.columns = newcolumns
    
    pca2 = PCA(n_components=validationmatrix.shape[1])
    pca2.fit(validationmatrix)
    principalComponents2 = pca2.fit_transform(validationmatrix)
    principalComponents2 = principalComponents2[:,:cutoff]
    principalDf2 = pd.DataFrame(data = principalComponents2)
    V2_new = principalDf2
    newcolumns2 = []
    for i in range(V2_new.shape[1]):
        string = 'feature' + str(i+1)
        newcolumns2.append(string)
    V2_new.columns = newcolumns2
    return X2_new,V2_new

In [ ]:
X2,V2 = drop_correlated(X,Xcorr,0.85,V)
X2_new,V2_new = do_PCA(X2,0.975,V2)

---

## Feature Selection using MDI

In [34]:
def featImpMDI(fit,featNames):
    # feat importance based on IS (in-sample) mean impurity reduction
    df0 = {i:tree.feature_importances_ for i,tree in enumerate(fit.estimators_)}
    df0 = pd.DataFrame.from_dict(df0,orient='index')
    df0.columns = featNames
    df0 = df0.replace(0,np.nan) # because max_features=1
    imp = pd.concat({'mean':df0.mean(),'std':df0.std()*df0.shape[0]**-.5},axis=1)
    imp /= imp['mean'].sum()
    return imp

In [56]:
def auxFeatImpSFI(featNames,clf,trnsX,y,cv):
    imp = pd.DataFrame(columns=['mean','std'])
    c = 0
    for featName in featNames:
        c += 1
        print(c)
        df0 = cross_val_score(clf, X = trnsX[[featName]], y = y, cv = cv, n_jobs = 4, verbose = 10)
        imp.loc[featName,'mean'] = df0.mean()
        imp.loc[featName,'std'] = df0.std()*df0.shape[0]**-.5
    return imp

In [35]:
def featImportance(X,y,clf,fit,method='SFI'):

#     clf = RandomForestClassifier(n_estimators=n_estimators,criterion='entropy',min_weight_fraction_leaf = minWLeaf,
#                                  max_features=int(1),class_weight='balanced',max_samples=max_samples,oob_score=True)
    #oob = fit.oob_score_
    if method == 'MDI':
        imp = featImpMDI(fit,X.columns)
    elif method == 'SFI':
        kf = KFold(n_splits=3, shuffle=False)
        imp = auxFeatImpSFI(X.columns,clf,X,y,kf)
        #scores = cross_val_score(classifier, X, y, cv=kf)
    return imp

In [59]:
clf2 = RandomForestClassifier(max_features=int(1))
fit = clf2.fit(X2,y)

In [60]:
mdi_imp = featImportance(X2,y,clf2,fit,method='MDI')

,mean,std
feature_intelligence1,0.009367,0.000175
feature_intelligence2,0.009569,0.000033
feature_intelligence4,0.010196,0.000109
feature_intelligence5,0.008898,0.000034
feature_intelligence6,0.008794,0.000037
...,...,...
feature_wisdom17,0.007801,0.000030
feature_wisdom18,0.007613,0.000026
feature_wisdom21,0.007765,0.000031
feature_wisdom23,0.007355,0.000035


In [62]:
#mdi_imp.to_excel('MDI Analysis - Main2.xlsx')

---

## Modeling using Random Forest, AdaBoost, XGBoost

In [20]:
def do_ML(trainingdata,traininglabels,testingdata,testinglabels,method):
    """
    Function that applies the Machine Learning method to the provided training and testing data and labels.
    
    Returns: ML model, out of sample AUC
    """
    if (method == 'AdaBoost'):
        clf=AdaBoostClassifier(n_estimators=100)
        fit=clf.fit(trainingdata,traininglabels)
        y_pred=clf.predict(testingdata)
        y_predproba = clf.predict_proba(testingdata)
        auc_score = roc_auc_score(testinglabels, y_predproba, multi_class='ovr')
    elif (method == 'Random Forest'):
        clf=RandomForestClassifier(n_estimators=100)
        fit=clf.fit(trainingdata,traininglabels)
        y_pred=clf.predict(testingdata)
        y_predproba = clf.predict_proba(testingdata)
        auc_score = roc_auc_score(testinglabels, y_predproba, multi_class='ovr')
    elif (method == 'XGBoost'):
        clf = XGBClassifier(n_estimators = 100, n_jobs=4)
        fit = clf.fit(trainingdata,traininglabels)
        preds = clf.predict(testingdata)
        predproba = clf.predict_proba(testingdata)
        auc_score = roc_auc_score(testinglabels, predproba, multi_class='ovr')
    return clf,fit,auc_score

In [17]:
#X2_new.to_csv("Features Matrix 2 - PCA.csv")

In [18]:
X_train, X_test, y_train, y__test = train_test_split(X2_new, y, test_size=0.3, random_state = 42)

In [ ]:
clfrf,fitrf,auc_scorerf = do_ML(X_train,y_train,X_test,y__test,method='Random Forest')
#print(auc_scorerf)
clfada,fitada,auc_scoreada = do_ML(X_train,y_train,X_test,y__test,method='AdaBoost')
#print(auc_scoreada)
clf,fit,auc_score = do_ML(X_train,y_train,X_test,y__test,method='XGBoost')
#print(auc_score)

In [149]:
filename = 'XGBoost - Training Only.sav'
pickle.dump(clf, open(filename, 'wb'))

In [35]:
clfrf,fitrf,auc_scorerf = do_ML(X2_new,y,V2_new,y_test,method='Random Forest')
#print(auc_scorerf)
clfada,fitada,auc_scoreada = do_ML(X2_new,y,V2_new,y_test,method='AdaBoost')
#print(auc_scoreada)
clf,fit,auc_score = do_ML(X2_new,y,V2_new,y_test,method='XGBoost')
#print(auc_score)

0.5705710904323456


In [36]:
filename = 'XGBoost - Training and Validation.sav'
pickle.dump(clf, open(filename, 'wb'))

---

## Hyper-parameter tuning using randomized search

In [ ]:
param_grid = {"n_estimators" :[100, 150, 200, 300], "learning_rate": [0.005,0.05,0.1,0.5],  "max_depth":[3,5,10]}
estimator = XGBClassifier(n_jobs = 4)
#gs = TuneSearchCV(estimator,param_distributions=param_grid,n_trials=3,early_stopping=True,search_optimization="random",max_iters=10)
gs2 = RandomizedSearchCV(estimator, param_distributions=param_grid, n_jobs=4, scoring='roc_auc_ovr', cv=3, verbose=10, n_iter = 3)
fit_gs = gs2.fit(X2_new,y)

In [25]:
filename2 = 'Randomized Search Result 2.sav'
pickle.dump(gs2, open(filename2, 'wb'))

In [31]:
filename = 'Randomized Search Result.sav'
pickle.dump(gs, open(filename, 'wb'))

In [33]:
gs.cv_results_, gs.best_params_, gs.best_score_

({'mean_fit_time': array([ 6179.7008427 ,  8741.79812074, 16565.43113446]),
  'std_fit_time': array([   9.52248098,  954.14269654, 2329.55769862]),
  'mean_score_time': array([ 9.21154928, 13.11428102, 25.28526306]),
  'std_score_time': array([1.55319174, 2.21975627, 8.18040702]),
  'param_n_estimators': masked_array(data=[150, 150, 150],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'param_max_depth': masked_array(data=[3, 5, 10],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'param_learning_rate': masked_array(data=[0.005, 0.05, 0.1],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'n_estimators': 150, 'max_depth': 3, 'learning_rate': 0.005},
   {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.05},
   {'n_estimators': 150, 'max_depth': 10, 'learning_rate': 0.1}],
  'split0_test_score': array([0.60774396, 0.632

In [24]:
gs2.cv_results_, gs2.best_params_, gs2.best_score_

({'mean_fit_time': array([12511.35207001,  7011.36263124, 12279.69900393]),
  'std_fit_time': array([  10.45473315, 1191.20844075, 1158.03233157]),
  'mean_score_time': array([19.05373271, 10.33086809, 15.06518459]),
  'std_score_time': array([0.45416106, 2.70567292, 4.60447042]),
  'param_n_estimators': masked_array(data=[300, 200, 100],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'param_max_depth': masked_array(data=[3, 3, 10],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'param_learning_rate': masked_array(data=[0.1, 0.05, 0.1],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.1},
   {'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.05},
   {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.1}],
  'split0_test_score': array([0.63702754, 0.6308778

In [29]:
gs1preds = gs.predict_proba(V2_new)
gs2preds = gs2.predict_proba(V2_new)

In [30]:
auc_scoregs1 = roc_auc_score(y_test, gs1preds, multi_class='ovr')
#print(auc_scoregs1)
auc_scoregs2 = roc_auc_score(y_test, gs2preds, multi_class='ovr')
#print(auc_scoregs2)

0.5794995035407928
0.575335700165294


---

## Numerai Test Set Predictions

In [11]:
def drop_correlated_w_Test(mainmatrix,correlationmatrix,correlationthreshold,testmatrix):
    correlated_features = set()
    for i in range(len(correlationmatrix.columns)):
        for j in range(i):
            if abs(correlationmatrix.iloc[i, j]) > correlationthreshold:
                colname = correlationmatrix.columns[i]
                correlated_features.add(colname)
    mainmatrix2 = mainmatrix.copy()
    mainmatrix2.drop(labels=correlated_features, axis=1, inplace=True)
    testmatrix2 = testmatrix.copy()
    testmatrix2.drop(labels=correlated_features, axis=1, inplace=True)
    return mainmatrix2,testmatrix2

In [12]:
def do_PCA_w_Test(matrix,explainedvariance,testmatrix):
    pca = PCA(n_components=matrix.shape[1])
    pca.fit(matrix)
    explainedvarianceratio = pca.explained_variance_ratio_
    cumulativevariance = np.cumsum(explainedvarianceratio)
    cutoff = np.searchsorted(cumulativevariance, explainedvariance, side='right')
    principalComponents = pca.fit_transform(matrix)
    principalComponents = principalComponents[:,:cutoff]
    principalDf = pd.DataFrame(data = principalComponents)
    X2_new = principalDf
    newcolumns = []
    for i in range(X2_new.shape[1]):
        string = 'feature' + str(i+1)
        newcolumns.append(string)
    X2_new.columns = newcolumns
    
    pca2 = PCA(n_components=testmatrix.shape[1])
    pca2.fit(testmatrix)
    principalComponents2 = pca2.fit_transform(testmatrix)
    principalComponents2 = principalComponents2[:,:cutoff]
    principalDf2 = pd.DataFrame(data = principalComponents2)
    T2_new = principalDf2
    newcolumns2 = []
    for i in range(T2_new.shape[1]):
        string = 'feature' + str(i+1)
        newcolumns2.append(string)
    T2_new.columns = newcolumns2
    return X2_new,T2_new

In [26]:
X2,T2 = drop_correlated_w_Test(X,Xcorr,0.85,T)
X2_new,T2_new = do_PCA_w_Test(X2,0.975,T2)

In [14]:
gs = pickle.load(open('Randomized Search Result.sav', 'rb'))

C:\Users\12027\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomizedSearchCV from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [27]:
gs1preds_test = gs.predict_proba(T2_new)

In [29]:
label_list = [0, 0.25, 0.50, 0.75, 1.0]

In [ ]:
gs1pred_scaled = numpy.multiply(gs1preds_test, label_list)

In [30]:
result = []
for i in range(len(gs1preds_test)):
    result.append(sum(np.multiply(gs1preds_test[i],label_list)))

In [32]:
test_set = test_set.set_index("id")

In [34]:
df_test = pd.DataFrame(index = test_set.index)
df_test['prediction'] = result

In [35]:
df_test.to_csv('predictions.csv')